<a href="https://colab.research.google.com/github/rooroogjoob/cis700_hw2/blob/master/action_castle_improved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Text adventure game

This Python notebook builds a simple text advenutre game inspired by the [Adventuron Classroom](https://adventuron.io/classroom/) design by Chris Ainsley of Adventuron Software Limited.

The main components are:
1. __The parser__, which interprets the player's commands.
2. __The game__, which represents the world (a collection of __locations__ and __items__), and describes what the player sees.
3. __The data__, which you input to create your own unique game.

## The Game Class
The game keeps track of the state of the world, and describes what the player sees as they move through different locations.

In [0]:
class Game:
  """The Game class represents the world.  Internally, we use a 
     graph of Location objects and Item objects, which can be at a 
     Location or in the player's inventory.  Each locations has a set of
     exits which are the directions that a player can move to get to an
     adjacent location. The player can move from one location to another
     location by typing a command like "Go North".
  """

  def __init__(self, start_at):
    # start_at is the location in the game where the player starts
    self.curr_location = start_at
    self.curr_location.has_been_visited = True
    # inventory is the set of objects that the player has collected/
    self.inventory = {}
    # Print the special commands associated with items in the game (helpful 
    # for debugging and for novice players).
    self.print_commands = True

  def describe(self):
    """Describe the current game state by first describing the current 
       location, then listing any exits, and then describing any objects
       in the current location."""
    self.describe_current_location()
    self.describe_exits()
    self.describe_items()

  def describe_current_location(self):
    """Describe the current location by printing its description field."""
    print(self.curr_location.description)

  def describe_exits(self):
    """List the directions that the player can take to exit from the current
       location."""
    exits = []
    for exit in self.curr_location.connections.keys():
      exits.append(exit.capitalize())
    if len(exits) > 0:
      print("Exits: ", end = '')
      print(*exits, sep = ", ",)
  
  def describe_items(self):
    """Describe what objects are in the current location."""
    if len(self.curr_location.items) > 0:
      print("You see: ")
      for item_name in self.curr_location.items:
        item = self.curr_location.items[item_name]
        print(item.description)
        if self.print_commands:
          special_commands = item.get_commands()
          for cmd in special_commands:
            print('\t', cmd)

  def add_to_inventory(self, item):
    """Add an item to the player's inventory."""
    self.inventory[item.name] = item
  
  def is_in_inventory(self,item):
    return item.name in self.inventory

  def get_items_in_scope(self):
    """Returns a list of items in the current location and in the inventory"""
    items_in_scope = []
    for item_name in self.curr_location.items:
      items_in_scope.append(self.curr_location.items[item_name])
    for item_name in self.inventory:
      items_in_scope.append(self.inventory[item_name])
    return items_in_scope

## Locations

Locations Locations are the places in the game that a player can visit.  They contain connects to other locations and items that the player can interact with.

In [0]:
class Location:
  """Locations are the places in the game that a player can visit.
     Internally they are represented nodes in a graph.  Each location stores
     a description of the location, any items in the location, its connections
     to adjacent locations, and any blocks that prevent movement to an adjacent
     location.  The connections is a dictionary whose keys are directions and
     whose values are the location that is the result of traveling in that 
     direction.  The travel_descriptions also has directions as keys, and its 
     values are an optional short desciption of traveling to that location.
  """
  def __init__(self, name, description, end_game=False):
    # A short name for the location
    self.name = name
    # A description of the location
    self.description = description
    # True if entering this location should end the game
    self.end_game = end_game
    # Dictionary mapping from directions to other Location objects
    self.connections = {}
    # Dictionary mapping from directions to text description of the path there
    self.travel_descriptions = {}
    # Dictionary mapping from item name to Item objects present in this location
    self.items = {}
    # Dictionary mapping from direction to Block object in that direction
    self.blocks = {}
    # Flag that gets set to True once this location has been visited by player
    self.has_been_visited = False

  def add_connection(self, direction, connected_location, travel_description=""):
    """Add a connection from the current location to a connected location.
       Direction is a string that the player can use to get to the connected
       location.  If the direction is a cardinal direction, then we also 
       automatically make a connection in the reverse direction."""
    self.connections[direction] = connected_location
    self.travel_descriptions[direction] = travel_description
    if direction == 'north':
      connected_location.connections["south"] = self
      connected_location.travel_descriptions["south"] = ""
    if direction == 'south':
      connected_location.connections["north"] = self
      connected_location.travel_descriptions["north"] = ""
    if direction == 'east':
      connected_location.connections["west"] = self
      connected_location.travel_descriptions["west"] = ""
    if direction == 'west':
      connected_location.connections["east"] = self
      connected_location.travel_descriptions["east"] = ""
    if direction == 'up':
      connected_location.connections["down"] = self
      connected_location.travel_descriptions["down"] = ""
    if direction == 'down':
      connected_location.connections["up"] = self
      connected_location.travel_descriptions["up"] = ""
    if direction == 'in':
      connected_location.connections["out"] = self
      connected_location.travel_descriptions["out"] = ""
    if direction == 'out':
      connected_location.connections["in"] = self
      connected_location.travel_descriptions["in"] = ""


  def add_item(self, name, item):
    """Put an item in this location."""
    self.items[name] = item

  def remove_item(self, item):
    """Remove an item from this location (for instance, if the player picks it
       up and puts it in their inventory)."""
    self.items.pop(item.name)


  def is_blocked(self, direction, game):
    """Check to if there is an obstacle in this direction."""
    if not direction in self.blocks:
        return False
    (block_description, preconditions) = self.blocks[direction]
    if check_preconditions(preconditions, game):
      # All the preconditions have been met.  You may pass.
      return False
    else: 
      # There are still obstalces to overcome or puzzles to solve.
      return True

  def get_block_description(self, direction):
    """Check to if there is an obstacle in this direction."""
    if not direction in self.blocks:
      return ""
    else:
      (block_description, preconditions) = self.blocks[direction]
      return block_description

  def add_block(self, blocked_direction, block_description, preconditions):
    """Create an obstacle that prevents a player from moving in the blocked 
       location until the preconditions are all met."""
    self.blocks[blocked_direction] = (block_description, preconditions)

## Checking Preconditions 
In text adventure games it's common to block a player's progress by creating blocks that prevent them from moving to a location.  For instance, a drawbridge might have a troll that you need to get rig of before you can cross into the castle, or a locked door might prevent you from entering a building until you have a key.  

This is a function that you can modify to include other preconditions.

In [0]:
def check_preconditions(preconditions, game, print_failure_reasons=True):
  """Checks whether the player has met all of the specified preconditions"""
  all_conditions_met = True
  for check in preconditions: 
    if check == "inventory_contains":
      item = preconditions[check]
      if not game.is_in_inventory(item):
        all_conditions_met = False
        if print_failure_reasons:
          print("You don't have the %s" % item.name)
    if check == "in_location":
      location = preconditions[check]
      if not game.curr_location == location:
        all_conditions_met = False
        if print_failure_reasons:
          print("You aren't in the correct location")
    if check == "location_has_item":
      item = preconditions[check]
      if not item.name in game.curr_location.items:
        all_conditions_met = False
        if print_failure_reasons:
          print("The %s isn't in this location" % item.name)
    # todo - add other types of preconditions
    if check == "location_does_not_have_item":
      item = preconditions[check]
      if item.name in game.curr_location.items:
        all_conditions_met = False
        if print_failure_reasons:
          print("The %s is still in this location" % item.name)
    if check == "has_crown":
      item = preconditions[check]
      if not game.is_in_inventory(item):
        all_conditions_met = False
        print("'You're not royalty!'")
        if print_failure_reasons:
          print("You don't have the %s" % item.name)



    
  return all_conditions_met

## Items
Items are objects that a player can get, or scenery that a player can examine. We could also implement people as items.  

In [0]:
class Item:
  """Items are objects that a player can get, or scenery that a player can
     examine."""
  def __init__(self,
               name,
               description,
               examine_text="",
               take_text="",
               start_at=None,
               gettable=True,
               end_game=False):
    # The name of the object
    self.name = name
    # The default description of the object.
    self.description = description
    # The detailed description of the player examines the object.
    self.examine_text = examine_text
    # Text that displays when player takes an object.
    self.take_text = take_text if take_text else ("You take the %s." % self.name)
    # Indicates whether a player can get the object and put it in their inventory.
    self.gettable = gettable
    # True if entering this location should end the game.
    self.end_game = end_game
    # The location in the Game where the object starts.
    if start_at:
      start_at.add_item(name, self)
    self.commands = {}


  def get_commands(self):
    """Returns a list of special commands associated with this object"""
    return self.commands.keys()

  def add_action(self, command_text, function, arguments, preconditions={}):
    """Add a special action associated with this item"""
    self.commands[command_text] = (function, arguments, preconditions)

  def do_action(self, command_text, game):
    """Perform a special action associated with this item"""
    end_game = False  # Switches to True if this action ends the game.
    if command_text in self.commands:
      function, arguments, preconditions = self.commands[command_text]
      if check_preconditions(preconditions, game):
        end_game = function(game, arguments)
    else:
      print("Cannot perform the action %s" % command_text)
    return end_game

In [5]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize

def get_senses(word):
  """Returns a list of word senses (WordNet synsets) for a word"""
  word_senses = wn.synsets(word)
  return word_senses

def get_definition(word_sense):
  return word_sense.definition()

def get_synonyms(word_sense):
  synonyms = []
  for lemma in word_sense.lemmas():
    synonym = lemma.name().replace('_', ' ')
    synonyms.append(synonym)
  return synonyms

#Here are the word senses for "bug". We can see what their distinct meanings are 
#by getting their definitions or their synonyms from WordNet.
word_senses = get_senses("bug")
for i, word_sense in enumerate(word_senses):
  print("\nSense %d: %s" % (i, word_sense.name()))
  print("Definition: ", get_definition(word_sense))
  print("Synonyms: ", get_synonyms(word_sense))

hyper = lambda s: s.hypernyms()
hypo = lambda s: s.hyponyms()

def get_hypernyms(word_sense, depth=5):
  return list(word_sense.closure(hyper, depth=depth))

def get_hyponyms(word_sense, depth=5):
  return list(word_sense.closure(hypo, depth=depth))

word_senses = get_senses("bug")
for i, word_sense in enumerate(word_senses):
  # The synset names include a word from the set of synonyms, 
  # plus a part of speech (n for noun, v for verb), and 
  # the number of the sense (sense 01 is the most common sense).
  print("\nSense %d: %s (%s)" % (i, word_sense.name(), get_definition(word_sense)))
  print("Hypernyms:")
  hypernyms = word_sense.hypernyms()
  while len(hypernyms) >0:
    print("%s\tis a\t%s" % (word_sense.name(), hypernyms[0].name()))
    word_sense = hypernyms[0]
    hypernyms = word_sense.hypernyms()

commands = [
	'wear crown',
	'smell rose',
	'eat fish',
	'light lamp',
	'give fish to troll',
	'propose to the princess',
	'go north',
]

def annotate_synsets(sentences):
  """This function queries WordNet for each word in a list of sentences,
     and asks the user to input a number corresponding to the synset."""

  word_senses = {}
  # Cached selections maps from word string to the previous
  # selection for this word (an integer)
  cached_selections = {}

  for i, sent in enumerate(sentences):
    words = word_tokenize(sent.lower())

    for word in words:
      sysnsets = wn.synsets(word)
      if len(sysnsets) != 0:
        selection = select_synset(sent, word, sysnsets, cached_selections)
        if selection != None:
          cached_selections[word] = selection
          if selection < len(sysnsets):
            s = sysnsets[selection]
            word_senses[word] = s.name()
  return word_senses


def select_synset(sent, word, sysnsets, cached_selections):
  """Ask the user to select which sense of the word  
     is being used in this sentence."""
  print(sent)
  print(word.upper())

  prev_selection = -1
  if word in cached_selections:
    prev_selection = cached_selections[word]

  for choice, s in enumerate(sysnsets):
    if choice == prev_selection:
      print("*** ", end = '')
    print("%d) %s - %s" % (choice, s.name(), s.definition()))

  choice += 1
  if choice == prev_selection:
    print("*** ", end = '')
  print("%d None of these." % choice)

  selection = -1
  while selection == -1:
    try:
      user_input = input(">")
      if user_input.strip() == 'x':
        # The user can press 'x' to exit.
        return None
      if user_input.strip() == '' and prev_selection > -1:
        # The user can press retrun to confirm the previous selection.
        return prev_selection
      selection = int(user_input)
    except:
      selection = -1
    if selection < 0 or selection > len(sysnsets):
      print("Please select a number between 0-%d, or type 'x' to exit" % len(sysnsets))
      if prev_selection > -1:
        print("You can also press return to confirm the previous selection (marked by ***).")
    else:
      return selection


def confirm_hyponyms(word, sysnset, do_hypernyms_instead=False):
  """Ask the user to confirm which of the hyponyms are applicable 
     for this sentence."""
  print(word.upper())

  confirmed = []
  if do_hypernyms_instead:
    unconfirmed = sysnset.hypernyms()
  else:
    unconfirmed = sysnset.hyponyms()

  while len(unconfirmed) > 0:
    s = unconfirmed.pop(0)
    print("Is %s an appropriate substitute for %s? (y/n)" % (s.name(), word))
    print("It means:", s.definition())
    print("Synonyms are:", get_synonyms(s))
    user_input = ''
    while user_input == '':
      user_input = input(">")
      user_input = user_input.strip()
      if user_input == 'y' or user_input == 'yes':
        confirmed.append(s.name())
        if do_hypernyms_instead:
          unconfirmed.extend(s.hypernyms())
        else:
          unconfirmed.extend(s.hyponyms())
        
      elif user_input == 'n' or user_input == 'no':
        pass
      elif user_input == 'x':
        # The user can press 'x' to exit.
        return confirmed
      else:
        print("Please type 'yes' or 'no' or 'x' to stop confirming for this word")
        user_input = ''
  return confirmed

# Save your annotations to a file, so that you can submit them with your homework.
def save_to_drive(word_senses, confirmed_hyponyms, confirmed_hypernyms):
  import json
  from google.colab import drive
  drive.mount('/content/drive/')

  output_file = '/content/drive/My Drive/word-sense-annotations.json'
  output_json = {}
  output_json['senses'] = word_senses
  output_json['hyponyms'] = confirmed_hyponyms
  output_json['hypernyms'] = confirmed_hypernyms

  with open(output_file, 'w') as write_file:
    write_file.write(json.dumps(output_json, sort_keys=True, indent=4))
    write_file.write('\n')



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.

Sense 0: bug.n.01
Definition:  general term for any insect or similar creeping or crawling invertebrate
Synonyms:  ['bug']

Sense 1: bug.n.02
Definition:  a fault or defect in a computer program, system, or machine
Synonyms:  ['bug', 'glitch']

Sense 2: bug.n.03
Definition:  a small hidden microphone; for listening secretly
Synonyms:  ['bug']

Sense 3: hemipterous_insect.n.01
Definition:  insects with sucking mouthparts and forewings thickened and leathery at the base; usually show incomplete metamorphosis
Synonyms:  ['hemipterous insect', 'bug', 'hemipteran', 'hemipteron']

Sense 4: microbe.n.01
Definition:  a minute life form (especially a disease-causing bacterium); the term is not in technical use
Synonyms:  ['microbe', 'bug', 'germ']

Sense 5: tease.v.01
Definition:  ann

In [0]:
import itertools #We're using the product method from itertools

def get_alternatives(word, word_senses, confirmed_hypernyms, confirmed_hyponyms):
  """Create a list of good alternatives for a word by listing out the synonyms
    for its word sense, and for its hyponyms and hypernyms."""
  alternatives = []
  if not word in word_senses:
    alternatives.append(word)
    return alternatives
  word_sense = wn.synset(word_senses[word])
  alternatives.extend(get_synonyms(word_sense))
  for hypernym in confirmed_hypernyms[word]:
    alternatives.extend(get_synonyms(wn.synset(hypernym)))
  for hyponym in confirmed_hyponyms[word]:
    alternatives.extend(get_synonyms(wn.synset(hyponym)))
  return alternatives

def enumerate_alternatives(sentence, word_senses, confirmed_hypernyms, confirmed_hyponyms):
  """Enumerate all of the sentenes that can result by taking any combination of
     the alternates for each word in the sentence."""
  words = word_tokenize(sentence.lower())
  # a list of lists
  alternatives_per_word = []
  for word in words:
    alternatives = get_alternatives(word, word_senses, confirmed_hypernyms, confirmed_hyponyms)
    alternatives_per_word.append(alternatives)
  
  alternative_to_original = {}
  # all combinations of a list of lists
  for words in list(itertools.product(*alternatives_per_word)):
    alt_sent = " ".join(words)
    alternative_to_original[alt_sent] = sentence
  return alternative_to_original




#alternative_commands = {}
#for command in commands:
 # alternative_commands.update(enumerate_alternatives(command, 
  #                                                   word_senses, 
   #                                                  confirmed_hypernyms, 
    #                                                 confirmed_hyponyms))

In [7]:
"""
word_senses = annotate_synsets(commands)
confirmed_hyponyms = {}
confirmed_hypernyms = {}
for word in word_senses:
  print("First, pick the word sense for the word '%s'" % word)
  print("==============")
  word_sense = wn.synset(word_senses[word])
  print("\nNext, pick which hypernyms of %s we should allow players to use." % word_sense.name())
  print("==============")
  confirmed_hypernyms[word] = confirm_hyponyms(word, word_sense, do_hypernyms_instead=True)
  print("\Finally, pick which hyponyms of %s we should allow players to use." % word_sense.name())
  print("==============")  
  confirmed_hyponyms[word] = confirm_hyponyms(word, word_sense)


print("You're done annotating!  Save your annotation to your Google drive.")
print("You need to paste in a confirmation code to allow Colab to have access.")
print("We'll create a file called 'word-sense-annotations.json' for you to turn in.")
print("==============")
save_to_drive(word_senses, confirmed_hyponyms, confirmed_hypernyms)
"""


'\nword_senses = annotate_synsets(commands)\nconfirmed_hyponyms = {}\nconfirmed_hypernyms = {}\nfor word in word_senses:\n  print("First, pick the word sense for the word \'%s\'" % word)\n  print("==============")\n  word_sense = wn.synset(word_senses[word])\n  print("\nNext, pick which hypernyms of %s we should allow players to use." % word_sense.name())\n  print("==============")\n  confirmed_hypernyms[word] = confirm_hyponyms(word, word_sense, do_hypernyms_instead=True)\n  print("\\Finally, pick which hyponyms of %s we should allow players to use." % word_sense.name())\n  print("==============")  \n  confirmed_hyponyms[word] = confirm_hyponyms(word, word_sense)\n\n\nprint("You\'re done annotating!  Save your annotation to your Google drive.")\nprint("You need to paste in a confirmation code to allow Colab to have access.")\nprint("We\'ll create a file called \'word-sense-annotations.json\' for you to turn in.")\nprint("==============")\nsave_to_drive(word_senses, confirmed_hyponyms,

In [0]:
import json

with open('/content/drive/My Drive/word-sense-annotations.json') as json_file:
    data = json.load(json_file)

ws = data['senses']
hyper = data['hypernyms']
hypo = data['hyponyms']


alternative_commands = {}
for command in commands:
  alternative_commands.update(enumerate_alternatives(command, 
                                                     ws, #word_senses, 
                                                     hyper, #confirmed_hypernyms, 
                                                     hypo)) #confirmed_hyponyms))

In [11]:
!pip3 install pymagnitude
!wget http://magnitude.plasticity.ai/glove/heavy/glove.6B.300d.magnitude
#!wget http://magnitude.plasticity.ai/word2vec/heavy/GoogleNews-vectors-negative300.magnitude

from pymagnitude import *
vectors = Magnitude("glove.6B.300d.magnitude")
#vectors = Magnitude("GoogleNews-vectors-negative300.magnitude")

from pymagnitude import *
vectors = Magnitude("glove.6B.300d.magnitude")

# FOR YOU TO DO: Write these function
def construct_sentence_vector(command, vectors):
  sentence_vector = np.zeros(shape=(vectors.dim,))
  for word in command.split():
    word_vector = vectors.query(word)
    # TODO - Do something
    sentence_vector += word_vector
  sentence_vector = sentence_vector / len(command.split())
  return sentence_vector

def find_most_similar_command(user_command, known_commands, vectors):
  # TODO - Do something
  commands = {}
  max_sim = 0
  most_sim = known_commands[0]

  user_vector = construct_sentence_vector(user_command, vectors)
  for command in known_commands:
    commands[command] = construct_sentence_vector(command, vectors)
  for command in commands:
    sim = vectors.similarity(commands[command], user_vector)
    if sim > max_sim:
      most_sim = command
      max_sim = sim
  return most_sim

     |████████████████████████████████| 5.4MB 7.8MB/s 
  Created wheel for pymagnitude: filename=pymagnitude-0.1.120-cp36-cp36m-linux_x86_64.whl size=135918205 sha256=514314407c9f19bc36319b4417fff5a430f314c575f8ce874bd1da8d1910e61a
  Stored in directory: /root/.cache/pip/wheels/a2/c7/98/cb48b9db35f8d1a7827b764dc36c5515179dc116448a47c8a1
Successfully built pymagnitude
--2020-01-30 03:37:06--  http://magnitude.plasticity.ai/glove/heavy/glove.6B.300d.magnitude
Resolving magnitude.plasticity.ai (magnitude.plasticity.ai)... 52.217.8.211
Connecting to magnitude.plasticity.ai (magnitude.plasticity.ai)|52.217.8.211|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1384890368 (1.3G) [binary/octet-stream]
Saving to: ‘glove.6B.300d.magnitude’

glove.6B.300d.magni 100%[===================>]   1.29G  35.2MB/s    in 38s     

2020-01-30 03:37:44 (35.0 MB/s) - ‘glove.6B.300d.magnitude’ saved [1384890368/1384890368]



## The Parser
The parser is the module that handles the natural language understanding in the game.  The players enter commands in text, and the parser interprets them and performs the actions that the player intends.  This is the module with the most potential for improvement using modern natural language processing.  The implementation that I have given below only uses simple keyword matching.

In [0]:
class Parser:
  """The Parser is the class that handles the player's input.  The player 
     writes commands, and the parser performs natural language understanding
     in order to interpret what the player intended, and how that intent
     is reflected in the simulated world. 
  """
  def __init__(self, game):
    # A list of all of the commands that the player has issued.
    self.command_history = []
    # A pointer to the game.
    self.game = game

  def get_player_intent(self,command):
    command = command.lower()
    if "," in command:
      # Let the player type in a comma separted sequence of commands
      return "sequence"
    elif self.get_direction(command):
      # Check for the direction intent
      return "direction"
    elif command.lower() == "look" or command.lower() == "l":
      # when the user issues a "look" command, re-describe what they see
      return "redescribe"
    elif "examine " in command or command.lower().startswith("x "):
      return "examine"
    elif  "take " in command or "get " in command:
      return "take"
    elif "drop " in command:
      return "drop"
    elif "inventory" in command or command.lower() == "i":
      return "inventory"
    else: 
      for item in self.game.get_items_in_scope():
        special_commands = item.get_commands()
        for special_command in special_commands:
          if command == special_command.lower():
            return "special"
      return "guess"

  def parse_command(self, command):
    # add this command to the history
    self.command_history.append(command)

    # By default, none of the intents end the game. The following are ways this
    # flag can be changed to True.
    # * Going to a certain place.
    # * Entering a certain special command
    # * Picking up a certain object.

    end_game = False

    # Intents are functions that can be executed
    intent = self.get_player_intent(command)
    if intent == "direction":
      end_game = self.go_in_direction(command)
    elif intent == "redescribe":
      self.game.describe()
    elif intent == "examine":
      self.examine(command)
    elif intent == "take":
      end_game = self.take(command)
    elif intent == "drop":
      self.drop(command)
    elif intent == "inventory":
      self.check_inventory(command)
    elif intent == "special":
      end_game = self.run_special_command(command)
    elif intent == "sequence":
      end_game = self.execute_sequence(command)
    elif intent == "guess":
      keys = []
      for key in alternative_commands.keys():
        keys.append(key)
      sim_command = find_most_similar_command(command, keys, vectors)
      new_command = alternative_commands[sim_command]
      #print(sim_command)
      #print(new_command)
      for item in self.game.get_items_in_scope():
        special_commands = item.get_commands()
      for special_command in special_commands:
        if new_command == special_command.lower():
          end_game = self.run_special_command(new_command)
    else:
      print("I'm not sure what you want to do.")
    return end_game

  ### Intent Functions ###

  def go_in_direction(self, command):
    """ The user wants to in some direction """
    direction = self.get_direction(command)

    if direction:
      if direction in self.game.curr_location.connections:
        if self.game.curr_location.is_blocked(direction, self.game):
          # check to see whether that direction is blocked.
          print(self.game.curr_location.get_block_description(direction))
        else:
          # if it's not blocked, then move there 
          self.game.curr_location = self.game.curr_location.connections[direction]

          # If moving to this location ends the game, only describe the location
          # and not the available items or actions.
          if self.game.curr_location.end_game:
            self.game.describe_current_location()
          else:
            self.game.describe()
      else:
        print("You can't go %s from here." % direction.capitalize())
    return self.game.curr_location.end_game

  def check_inventory(self,command):
    """ The player wants to check their inventory"""
    if len(self.game.inventory) == 0:
      print("You don't have anything.")
    else:
      descriptions = []
      for item_name in self.game.inventory:
        item = self.game.inventory[item_name]
        descriptions.append(item.description)
      print("You have: ", end = '')
      print(*descriptions, sep = ", ",)
  

  def examine(self, command):
    """ The player wants to examine something """
    command = command.lower()
    matched_item = False
    # check whether any of the items at this location match the command
    for item_name in self.game.curr_location.items:
      if item_name in command:
        item = self.game.curr_location.items[item_name]
        if item.examine_text:
          print(item.examine_text)
          matched_item = True
        break
    # check whether any of the items in the inventory match the command
    for item_name in self.game.inventory:
      if item_name in command:
        item = self.game.inventory[item_name]
        if item.examine_text:
          print(item.examine_text)
          matched_item = True
    # fail
    if not matched_item:
      print("You don't see anything special.")


  def take(self, command):
    """ The player wants to put something in their inventory """
    command = command.lower()
    matched_item = False

    # This gets set to True if posession of this object ends the game.
    end_game = False

    # check whether any of the items at this location match the command
    for item_name in self.game.curr_location.items:
      if item_name in command:
        item = self.game.curr_location.items[item_name]
        if item.gettable:
          self.game.add_to_inventory(item)
          self.game.curr_location.remove_item(item)
          print(item.take_text)
          end_game = item.end_game
        else:
          print("You cannot take the %s." % item_name)
        matched_item = True
        break
    # check whether any of the items in the inventory match the command
    if not matched_item:
      for item_name in self.game.inventory:
        if item_name in command:
          print("You already have the %s." % item_name)
          matched_item = True
    # fail
    if not matched_item:
      print("You can't find it.")

    return end_game

  def drop(self, command):
    """ The player wants to remove something from their inventory """
    command = command.lower()
    matched_item = False
    # check whether any of the items in the inventory match the command
    if not matched_item:
      for item_name in self.game.inventory:
        if item_name in command:
          matched_item = True
          item = self.game.inventory[item_name]
          self.game.curr_location.add_item(item_name, item)
          self.game.inventory.pop(item_name)
          print("You drop the %s." % item_name)
          break
    # fail
    if not matched_item:
      print("You don't have that.")


  def run_special_command(self, command):
    """Run a special command associated with one of the items in this location
       or in the player's inventory"""
    for item in self.game.get_items_in_scope():
      special_commands = item.get_commands()
      for special_command in special_commands:
        if command == special_command.lower():
          return item.do_action(command, self.game)

  def execute_sequence(self, command):
    for cmd in command.split(","):
      cmd = cmd.strip()
      self.parse_command(cmd)

  def get_direction(self, command):
    command = command.lower()
    if command == "n" or "north" in command:
      return "north" 
    if command == "s" or "south" in command:
      return "south"
    if command == "e" or "east" in command: 
      return "east"
    if command == "w" or "west" in command:
      return "west"
    if command == "up":
      return "up"
    if command == "down":
      return "down"
    if command.startswith("go out"):
      return "out"
    if command.startswith("go in"):
      return "in"
    for exit in self.game.curr_location.connections.keys():
      if command == exit.lower() or command == "go " + exit.lower():
        return exit
    return None

## Special functions
Many times we want to add special behavior to items in the game.  For instance, we might want to be able to _pick a rose_ from a _rosebush_, or the _eat_ a _fish_.  In this implementation we do this in a pretty generic way by allowing the game developer to call ```Item.add_action(cmd,function,argment,preconditions)``` where ```function``` is any Python function. Some example of functions are defined below.

These functions should return True if the game is ended by the action, False otherwise.

In [0]:
def add_item_to_inventory(game, *args):
  """ Add a newly created Item and add it to your inventory."""
  (item, action_description, already_done_description) = args[0]
  if(not game.is_in_inventory(item)):
    print(action_description)
    game.add_to_inventory(item)
  else:
    print(already_done_description)
  return False

def describe_something(game, *args):
  """Describe some aspect of the Item"""
  (description) = args[0]
  print(description)
  return False

def destroy_item(game, *args):
  """Removes an Item from the game by setting its location is set to None."""
  (item, action_description) = args[0]
  if game.is_in_inventory(item):
    game.inventory.pop(item.name)
    print(action_description)
  elif item.name in game.curr_location.items:
    game.curr_location.remove_item(item)
    print(action_description)
  return False

def create_item(game, *args):
  (item, action_description) = args[0]
  game.curr_location.add_item(item.name, item)
  print(action_description)
  return False

def destroy_items(game, *args):
  for item in args[0]:
    destroy_item(game, item)
  return False

def perform_multiple_actions(game, *args):
  for action_pair in args[0]:
    (action, action_args) = action_pair
    action(game, action_args)
  return False

def create_item_in_loc(game, *args):
  (item, location) = args[0]
  current_location = game.curr_location
  game.curr_location = location
  game.curr_location.add_item(item.name, item)
  game.curr_location = current_location
  return False

def destroy_item_in_loc(game, *args):
  (item, location) = args[0]
  current_location = game.curr_location
  game.curr_location = location
  destroy_item(game, (item, ""))
  game.curr_location = current_location
  return False


def end_game(game, *args):
  """Ends the game."""
  end_message = args[0]
  print(end_message)
  return True

## Game Data

Here's where you can define the locations and items in your game.  To get you started, I defined a super-simple fishing game, which contains the first 3 locations of __Action Castle__ by Jared A. Sorensen, which is part of the awesome book [Parsley](http://www.memento-mori.com/parsely-products/parsely-pdf).  

You can play through the whole game with the following commands:
1. take pole
2. go out
3. south 
4. catch fish with pole
5. eat fish

In [0]:
def build_game():
  # Locations
  cottage = Location("Cottage", "You are standing in a small cottage.")
  garden_path = Location("Garden Path", "You are standing on a lush garden path. There is a cottage here.")
  cliff = Location("Cliff", "There is a steep cliff here. You fall off the cliff and lose the game. THE END.", end_game=True)
  fishing_pond = Location("Fishing Pond", "You are at the edge of a small fishing pond.")
  winding_path = Location("Winding Path", "You are walking along a winding path. There is a tall tree here.")
  top_of_tall_tree = Location("Top Of Tall Tree", "You are the top of the tall tree. There is a stout, dead branch here.")
  draw_bridge = Location("Draw Bridge", "You are standing on one side of a drawbridge leading to ACTION CASTLE. There is a mean troll here.")
  court_yard = Location("Court Yard", "You are in the courtyard of ACTION CASTLE.")
  feasting_hall = Location("Great Feasting Hall", "You stand inside the Great Feasting Hall. There is a strange candle here.")
  tower_stairs = Location("Tower Stairs", "You are climbing the stairs to the tower. There is a locked door here.")
  dungeon_stairs = Location("Dungeon Stairs", "You are climbing the stairs down to the dungeon. It is too dark to see!")
  tower = Location("Tower", "You are inside a tower. The princess is here.")
  dungeon = Location("Dungeon", "You are in the dungeon. There is a spooky ghost here.")
  throne_room = Location("Throne Room", "This is the throne room of ACTION CASTLE. There is an ornate golden throne here.")

  # Connections
  cottage.add_connection("out", garden_path)
  garden_path.add_connection("west", cliff)
  garden_path.add_connection("north", winding_path)
  garden_path.add_connection("south", fishing_pond)
  winding_path.add_connection("up", top_of_tall_tree)
  winding_path.add_connection("east", draw_bridge)
  draw_bridge.add_connection("east", court_yard)
  court_yard.add_connection("east", feasting_hall)
  court_yard.add_connection("up", tower_stairs)
  court_yard.add_connection("down", dungeon_stairs)
  tower_stairs.add_connection("up", tower)
  dungeon_stairs.add_connection("down", dungeon)
  feasting_hall.add_connection("east", throne_room)

  # Items that you can pick up
  fishing_pole = Item("pole", "a fishing pole", "A SIMPLE FISHING POLE.", start_at=cottage)
  potion = Item("potion", "a poisonous potion", "IT'S BRIGHT GREEN AND STEAMING.", start_at=cottage, take_text='As you near the potion, the fumes cause you to faint and lose the game. THE END.', end_game=True)
  rosebush = Item("rosebush", "a rosebush", "THE ROSEBUSH CONTAINS A SINGLE RED ROSE.  IT IS BEAUTIFUL.", start_at=garden_path)
  rose = Item("rose", "a red rose", "IT SMELLS GOOD.",  start_at=None)
  fish = Item("fish", "a dead fish", "IT SMELLS TERRIBLE.", start_at=None)
  branch = Item("branch", "a dead branch", start_at=top_of_tall_tree)
  key = Item("key", "a key", "IT IS VERY SHINY.", start_at=None)
  unlit_lamp = Item("lamp (unlit)", "a lamp (unlit)", "IT CAN LIGHT UP THE DARK. IT IS CURRENTLY OUT.", start_at=None)
  lit_lamp = Item("lamp (lit)", "a lamp (lit)", "IT IS SHINING BRIGHT!", start_at=None)
  candle = Item("candle", "a strange candle", "IT IS COVERED IN STRANGE RUINS", start_at=feasting_hall)
  crown = Item("crown", "a crown", "IT IS VERY SHINY.", start_at=None)
  wedding_ring = Item("wedding ring", "your wedding ring", "YOU'RE MARRIED TO THE PRINCESS", start_at=None)

  # Sceneary (not things that you can pick up)
  pond = Item("pond", "a small fishing pond", "THERE ARE FISH IN THE POND.", start_at=fishing_pond, gettable=False)
  long_fall = Item("long fall", "a long fall", "LOOKS LIKE FUN", start_at=top_of_tall_tree, gettable=False)
  troll = Item("troll", "a salivating troll", "THE TROLL IS WARTY, GREEN, AND, HUNGRY", start_at=draw_bridge, gettable=False)
  guard = Item("guard", "a guard carrying a sword and a key", "HE LOOKS AT YOU SUSPICIOUSLY.",
    start_at=court_yard, gettable=False)
  unconscious_guard = Item("unconscious guard", "an unconscious guard is slumpped against the wall", 
    "HE HAS BITS OF BRANCH ON HIS UNIFORM.", start_at=None, gettable=False)
  ghost = Item("ghost", "The ghost has bony, claw-like fingers and wears a crown.", start_at=dungeon, gettable=False)
  initial_princess = Item("princess", "The princess is beautiful, sad and lonely. She awaits her prince.", start_at=tower, gettable=None)
  princess = Item("princess", "The princess is beautiful, sad and lonely. She awaits her prince.", start_at=None, gettable=None)
  queen = Item("princess (now your queen)", "The princess is beautiful, happy and has her king.", start_at=None, gettable=None)
  throne = Item("throne", "An ornate gold throne.", start_at=throne_room, gettable=False)

  new_guard = Item("happy guard", "a guard carrying a sword and a key. He kneels to hail his new king.", "HE KNEELS TO HAILS HIS NEW KING", start_at=None, gettable=False)
  revelers = Item("revelers", "courtiers, guards and other subjects celebrating", "THEY'RE CELEBRATING YOU!", start_at=None, gettable=False)

  #Blocks
  draw_bridge.add_block("east", "The troll blocks your way. Spittle falls from its mouth.", preconditions={"location_does_not_have_item": troll})
  court_yard.add_block("east", "The guard blocks your path east.", preconditions={"location_does_not_have_item": guard})
  tower_stairs.add_block("up", "The door is locked.", preconditions={"inventory_contains": key})
  dungeon_stairs.add_block("down", "You cannot see in order to find your way safely down", preconditions={"inventory_contains": lit_lamp})

  # Add special functions to your items
  rosebush.add_action("pick rose",  add_item_to_inventory, (rose, "You pick the lone rose from the rosebush.", "You already picked the rose."))
  rose.add_action("smell rose",  describe_something, ("It smells sweet."))
  pond.add_action("catch fish",  describe_something, ("You reach into the pond and try to catch a fish with your hands, but they are too fast."))
  pond.add_action("catch fish with pole",  add_item_to_inventory, (fish, "You dip your hook into the pond and catch a fish.","You weren't able to catch another fish."), preconditions={"inventory_contains":fishing_pole})
  fish.add_action("eat fish", describe_something, ("That's disgusting! It's raw! And definitely not sashimi-grade! You elect not to eat it."))
  branch.add_action("snap off branch", add_item_to_inventory, (branch, "You snap off the dead branch.", "You already broke off the branch."))
  long_fall.add_action("jump", end_game, ("You jump off the tree and break your legs. You do not die, until you suddenly do."))
  troll.add_action("attack troll", end_game, ("You attack the troll with your bare hands. However, the troll catches your fists mid-swing. Luckily for you, the troll is a pescatarian. Unluckily for you, the troll is not a pacifist."))
  troll.add_action("attack troll with branch", end_game, ("You attack the troll with the branch. The branch shatters on the troll's head. He is unaffected. Luckily for you, the troll is a pescatarian. Unluckily for you, the troll is not a pacifist."), preconditions={"inventory_contains": branch})
  troll.add_action("kiss the troll", end_game, ("The troll blushes. You soon get married and live together for many years before the inevitable divorce and you eventually die alone since your half-troll children despise you."))
  troll.add_action("give fish to troll", destroy_items, 
                   ((fish, "The troll seems satisfied after devouring the fish."), (troll, "He stomps away happy about making a friend today. He cannot wait to tell his mother, who is also a troll.")), preconditions={"inventory_contains":fish})
  troll.add_action("feed fish to troll", destroy_items, 
                   ((fish, "The troll seems satisfied after devouring the fish."), (troll, "He stomps away happy about making a friend today. He cannot wait to tell his mother, who is also a troll.")), preconditions={"inventory_contains":fish})
  guard.add_action("hit guard with branch", perform_multiple_actions, 
      ([(destroy_item, (branch,"You swing your branch against the guard. It shatters to pieces.")),
      (destroy_item, (guard,"The guard slumps over, unconscious.")),
      (create_item, (unconscious_guard,"The guard's unconscious body lies on the ground.")),
      (create_item, (key,"His key falls from his hand.")),
      ]), preconditions={"inventory_contains":branch , "location_has_item": guard})
  unlit_lamp.add_action("light lamp", perform_multiple_actions, 
                        ([(destroy_item, (unlit_lamp, "You light the lamp.")), 
                        (add_item_to_inventory, (lit_lamp, "", ""))
                        ]), preconditions={"inventory_contains": unlit_lamp})
  lit_lamp.add_action("put out lamp", perform_multiple_actions, 
                        ([(destroy_item, (lit_lamp, "You put out the lamp.")), 
                        (add_item_to_inventory, (unlit_lamp, "", ""))
                        ]), preconditions={"inventory_contains": lit_lamp})
  candle.add_action("light candle", perform_multiple_actions, 
                    ([(destroy_item, (candle, "You light the candle. It gives off a strange, acrid-smelling smoke.")),
                      (destroy_item, (ghost, "The smoke causes the ghost to drop its crown and flee.")),
                      (create_item, (crown, ""))
                    ]), preconditions={"in_location": dungeon})
  candle.add_action("read runes", describe_something, ("The runes seem to be a spell of exorcism."))
  initial_princess.add_action("talk to princess", describe_something, ("She seems to be ignoring you while she longingly looks out the window to a field of flowers."))
  initial_princess.add_action("give rose to princess", perform_multiple_actions,
                             ([(destroy_item, (initial_princess, "She brightens up immediately!")), 
                               (destroy_item, (rose, "")),
                               (create_item, (princess, ""))
                             ]), preconditions={"inventory_contains": rose})
  princess.add_action("kiss princess", describe_something, ("'Not until we're wed!'"))
  princess.add_action("be rude to princess", describe_something, ("The princess slaps you."))
  princess.add_action("talk to princess about ghost", describe_something, ("'My father haunts the dungeon as a restless spirit.'"))
  princess.add_action("talk to princess about crown", describe_something, ("'Only the rightful heir to the throne can wear it!'"))
  princess.add_action("talk to princess about herself", describe_something, ("'I cannot leave this tower until I am married!'"))
  princess.add_action("talk to princess about throne", describe_something, ("'Only the king can sit on the throne.'"))
  princess.add_action("propose to princess", perform_multiple_actions,
                      ([(describe_something, ("You propose to the princess. \n 'My father’s crown! You have put his soul at rest and may now succeed him!' \n You and the princess are now married. She places the crown on your head.")),
                      (destroy_item, (crown, "")), 
                      (destroy_item, (princess, "")), 
                      (create_item, (queen, "")),
                      (add_item_to_inventory, (wedding_ring, "", "")),
                      (create_item_in_loc, (new_guard, court_yard)),
                      (create_item_in_loc, (revelers, feasting_hall)),
                      (create_item_in_loc, (revelers, throne_room)),
                      (destroy_item_in_loc, (unconscious_guard, court_yard))
                      ]), preconditions={"has_crown": crown})
  princess.add_action("marry princess", perform_multiple_actions,
                      ([(describe_something, ("You propose to the princess. \n 'My father’s crown! You have put his soul at rest and may now succeed him!' \n You and the princess are now married. She places the crown on your head.")),
                      (destroy_item, (crown, "")), 
                      (destroy_item, (princess, "")), 
                      (create_item, (queen, "")),
                      (add_item_to_inventory, (wedding_ring, "", "")),
                      (create_item_in_loc, (new_guard, court_yard)),
                      (create_item_in_loc, (revelers, feasting_hall)),
                      (create_item_in_loc, (revelers, throne_room)),
                      (destroy_item_in_loc, (unconscious_guard, court_yard))
                      ]), preconditions={"has_crown": crown})
  throne.add_action("sit on throne", end_game, ("You sit on the ornate golden throne. The people cheer for the new ruler of... ACTION CASTLE!"), preconditions={"inventory_contains": wedding_ring})                

  game = Game(cottage)
  return game


# Play the game
This small snippet of code is what you need to run the game.  Behold! The magestic prompt! 

In [0]:
def game_loop():
  game = build_game()
  parser = Parser(game)
  game.describe()

  command = ""
  while not (command.lower() == "exit" or command.lower == "q"):
    command = input(">")
    end_game = parser.parse_command(command)
    if end_game:
      return

game_loop()
print('THE GAME HAS ENDED.')

# Visualize your game
The code below allows you to create a directed graph that shows the locations in your game and how they are connected.  You can also save a PDF of your graph to your Google Drive with the `save_to_drive` method.  The output file will be called `game-visualization.pdf`.

In [0]:
#!pip install graphviz
from graphviz import Digraph
from IPython.display import Image
import queue

def DFS(game, graph):
  """Do a depth-first-search traversal of the locations in the game
     starting at the start location, and create a GraphViz graph 
     to vizualize the connections between the locations, and the items
     that are located at each location."""
  start_location = game.curr_location
  frontier = queue.Queue()
  frontier.put(start_location)
  visited = {}
  visited[start_location.name] = True

  while not frontier.empty():
    current_location = frontier.get()
    game.curr_location = current_location
    name = current_location.name
    description = current_location.description
    items = current_location.items
    items_html = describe_items(current_location)
    html = "<<b>%s</b><br />%s<br />%s>" % (name, description, items_html)
    # Create a new node in the graph for this location
    graph.node(name, label=html)  

    connections = current_location.connections
    for direction in connections.keys():
      next_location = connections[direction]
      if not current_location.is_blocked(direction, game):
        # Create an edge between the current location and its successor
        graph.edge(name, next_location.name, label=direction.capitalize())
      else:
        # Create a dotted edge for connected locations that are blocked
        block_description = "%s\n%s" % (direction.capitalize(), current_location.get_block_description(direction))
        graph.edge(name, next_location.name, label=block_description, style="dotted")
      if not next_location.name in visited:
        visited[next_location.name] = True
        frontier.put(next_location)

def describe_items(location, print_commands=True):
    """Describe what objects are in the current location."""
    items_html = ""
    if len(location.items.keys()) > 0:
      items_html = "You see: "
    for item_name in location.items:
      item = location.items[item_name]
      items_html += item.description
      if print_commands:
        special_commands = item.get_commands()
        for cmd in special_commands:
          items_html += "<br/><i>%s</i>" % cmd
    return items_html

def save_to_drive(graph):
  from google.colab import drive
  drive.mount('/content/drive/')
  graph.render('/content/drive/My Drive/game-visualization', view=True)  

graph = Digraph(node_attr={'color': 'lightblue2', 'style': 'filled'})
game = build_game()
DFS(game, graph)
save_to_drive(graph)
graph
